In [72]:
import pandas as pd
import numpy as np
from stage1 import getPrediction
import datetime


def alpha_model():
    def runAlpha():
        
        df_3_months = getPrediction()
        # 한달만

        # 포트폴리오 생성을 위한 자산 리스트
        assets = df_3_months.columns

        # 포트폴리오 생성 횟수
        num_portfolios = 10000

        port_ratios = []  # 포트폴리오 비중 리스트
        port_returns = np.array([])  # 연간 수익률 배열
        port_risks = np.array([])  # 연간 리스크(변동성) 배열


        
        for i in range(num_portfolios):
            # 무작위로 포트폴리오 비중 생성
            weights = np.random.random(len(assets))
            weights /= np.sum(weights)  # 총합을 1로 만들어줌

            portfolio_returns = np.sum(weights * df_3_months, axis=1)

            # 포트폴리오의 3개월간 누적 수익률 계산
            cumulative_returns = np.prod(1 + portfolio_returns) - 1

            # 포트폴리오 리스크 계산 (간단하게 표준편차를 사용)
            cov_matrix = df_3_months.cov()
            portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
            portfolio_volatility = np.sqrt(portfolio_variance)

            port_ratios.append(weights)
            port_returns = np.append(port_returns, cumulative_returns)
            port_risks = np.append(port_risks, portfolio_volatility)


        # 결과 데이터프레임 생성
        portfolio_data = {'Returns': port_returns, 'Risk': port_risks}
        for i, asset in enumerate(assets):
            portfolio_data[asset] = [weights[i] for weights in port_ratios]

        portfolio_df = pd.DataFrame(portfolio_data)


        # 최대 샤프 지수를 갖는 포트폴리오 선택
        max_sharpe_portfolio = portfolio_df.iloc[portfolio_df['Returns'].idxmax()]
        return max_sharpe_portfolio


    def wish_date_weight(): # date는 '$$$$-$$-$$'형식으로 받아온다
        result = runAlpha()
        stable_asset = ['kor3y','kor10y','us3y','us10y','gold']
        risky_asset = ['us', 'uk' ,'jp',	'euro',	 'ind',	'tw',	'br','kor']

        stable_weight = pd.DataFrame(columns=['kor3y','kor10y','us3y','us10y','gold'])
        stable_sum = 0
        risky_weight = pd.DataFrame(columns=['us', 'uk' ,'jp',	'euro',	 'ind',	'tw',	'br','kor'])
        risky_sum = 0

        row = result.copy()
        stable_sum = row[stable_asset].sum()
        risky_sum =  row[risky_asset].sum()

        final = row.copy() #result 원본값을 그대로 보존하기 위해서
        if stable_sum<0.4:

            risky_weight= row.copy() # 모델의 가중치가 곧 공격형 자산비중
            # print( risky_weight - result)
            final[stable_asset] = row[stable_asset]*(0.4/stable_sum)

            final[risky_asset] = row[risky_asset]*(0.6/risky_sum)
            
            stable_weight = final # 수정한 비중으로 안전형 자산

        elif stable_sum>=0.4: 

            stable_weight = row.copy() # 모델의 가중치가 곧 안전형 자산 비중

            final[stable_asset] = row[stable_asset]*(0.4/stable_sum)
            
            final[risky_asset] = row[risky_asset]*(0.6/risky_sum)
            
            risky_weight = final # 수정한 비중으로 공격형 자산
        return stable_weight, risky_weight

    stable_weight , risky_weight = wish_date_weight()
    stable_weight = stable_weight.astype('float')
    stable_weight['type'] = "A/안정형"
  
    risky_weight = risky_weight.astype('float')
    risky_weight['type'] = "A/공격형"
    return stable_weight[2:], risky_weight[2:]


In [106]:
test.set_index('date',inplace=True)

In [129]:
from tqdm.auto import tqdm
def make_data(df_, lookback_window, lookahead_window):
        df = df_.copy()

        # interleave로 해봐도 될 듯
        # data with returns
        # data_w_ret = np.concatenate([ df.values[1:], df.pct_change().values[1:] ], axis=1)

        # df = df.iloc[1:] #.values

        data_w_ret = np.dstack([df.values[1:], df.pct_change().values[1:]])
        df = df.iloc[1:] #.values
        data_w_ret = data_w_ret.reshape(df.shape[0],-1)

        # total_sample_num = df.shape[0]-window_size+1
        total_sample_num = df.shape[0]-(lookback_window+lookahead_window)+1 # lookback을 위한, lookahead를 위한 두 개 빼주기

        x_data = np.zeros((total_sample_num, lookback_window, data_w_ret.shape[1]))
        # unsupervised 방식이지만 sharpe에서 활용하도록 작성
        y_data = np.zeros((total_sample_num, lookahead_window, df.shape[1]))

        portfolio_date = []
        # for idx in tqdm(range(total_sample_num)):
        # standard_day를 기준으로 -lookback_window만큼 뒤를 보고, lookahead_window만큼 앞을 보기 위해 for문 조정
        # 예전 idx를 standard_day로 바꾼 것
        for idx, standard_day in tqdm(enumerate(range(lookback_window, total_sample_num+lookback_window)), total=total_sample_num):
        
            # x_data[idx,] = data_w_ret[idx:idx+window_size]
            x_data[idx,] = data_w_ret[standard_day-lookback_window:standard_day]

            y_data[idx,] = df[standard_day:standard_day+lookahead_window]

            portfolio_date.append(df.index[standard_day])

            # portfoilo_day.append

        return x_data, y_data, portfolio_date

In [130]:
lookback_window = 50
lookahead_window = 30
x_test, y_test, portfolio_test_date= make_data(test, lookback_window, lookahead_window)

100%|██████████| 166/166 [00:00<00:00, 9786.96it/s]


In [131]:
x_test

array([[[ 2989.24    ,     0.000157, 15622.72    , ...,     0.014003,
         59855.929688,     0.011367],
        [ 2953.97    ,    -0.011799, 15100.17    , ...,    -0.001424,
         60223.148438,     0.006135],
        [ 2920.53    ,    -0.01132 , 15080.87    , ...,    -0.007137,
         59601.839844,    -0.010317],
        ...,
        [ 2694.51    ,     0.013267, 13614.78    , ...,     0.029951,
         57863.929688,     0.018433],
        [ 2707.02    ,     0.004643, 13893.84    , ...,     0.000476,
         57578.208984,    -0.004938],
        [ 2686.05    ,    -0.007747, 13838.46    , ...,     0.005948,
         57292.488281,    -0.004962]],

       [[ 2953.97    ,    -0.011799, 15100.17    , ...,    -0.001424,
         60223.148438,     0.006135],
        [ 2920.53    ,    -0.01132 , 15080.87    , ...,    -0.007137,
         59601.839844,    -0.010317],
        [ 2954.89    ,     0.011765, 14935.9     , ...,    -0.010788,
         59744.648438,     0.002396],
        ...,


In [117]:
df = pd.read_csv("total_17_22.csv",index_col=0)

test = df[(df.index>=('2022'))]

In [124]:
def wish_date_weight(post_prediction,date): 
    
        stable_asset = ['kor3y','kor10y','us3y','us10y','gold']
        risky_asset = ['us', 'uk' ,'jp',	'euro',	 'ind',	'tw',	'br','kor']
            
        stable_weight = pd.DataFrame(columns=['kor3y','kor10y','us3y','us10y','gold'])
        stable_sum = 0
        risky_weight = pd.DataFrame(columns=['us', 'uk' ,'jp',	'euro',	 'ind',	'tw',	'br','kor'])
        risky_sum = 0

        # date의 행을 선택
        row = post_prediction[post_prediction['date']==date]
        stable_sum = row[stable_asset].sum(axis=1).values[0]
        risky_sum =  row[risky_asset].sum(axis=1).values[0]

        final = row.copy() #post_prediction 원본값을 그대로 보존하기 위해서
        if stable_sum<0.4:
        
            risky_weight.loc[date] = row.copy() # 모델의 가중치가 곧 공격형 자산비중
            # print( risky_weight - result)
            final[stable_asset] = row[stable_asset]*(0.4/stable_sum)

            final[risky_asset] = row[risky_asset]*(0.6/risky_sum)
            
            stable_weight = final.loc[date] # 수정한 비중으로 안전형 자산

        elif stable_sum>=0.4: 
        
            stable_weight = row.copy() # 모델의 가중치가 곧 안전형 자산 비중

            final[stable_asset] = row[stable_asset]*(0.4/stable_sum)
            
            final[risky_asset] = row[risky_asset]*(0.6/risky_sum)
            
            risky_weight = final # 수정한 비중으로 공격형 자산

        return stable_weight, risky_weight

In [115]:
test

,kospi,nasdaq,euro_stoxx,ftse,nikkei,korea_bond_03,korea_bond_10,america_bond_03,america_bond_10,gold,brazil,taiwan,india
date,,,,,,,,,,,,,
2022-01-03,2988.77,15832.800,4331.82,7454.08,29046.75,111700.0,68415.0,85.440,113.930,1800.85,103922.0,18270.509766,59183.218750
2022-01-04,2989.24,15622.720,4367.62,7505.15,29301.79,111635.0,68450.0,85.490,113.870,1814.56,103514.0,18526.349609,59855.929688
2022-01-05,2953.97,15100.170,4392.15,7516.87,29332.16,111485.0,68030.0,85.380,113.390,1809.53,101006.0,18499.960938,60223.148438
2022-01-06,2920.53,15080.870,4324.81,7450.37,28487.87,111160.0,67480.0,85.310,113.110,1788.68,101561.0,18367.919922,59601.839844
2022-01-07,2954.89,14935.900,4305.83,7485.28,28478.56,111090.0,67460.0,85.290,112.740,1795.63,102719.0,18169.759766,59744.648438
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,2313.69,10497.860,3817.01,7473.01,26235.25,107610.0,63150.0,81.320,96.750,1797.91,109698.0,14271.629883,59845.289062
2022-12-26,2317.14,10425.545,3824.95,7481.07,26405.87,107545.0,62750.0,81.265,96.345,1797.61,108738.0,14285.129883,60566.421875
2022-12-27,2332.79,10353.230,3832.89,7489.13,26447.87,107555.0,62840.0,81.210,95.940,1813.93,108347.0,14328.429688,60927.429688


In [116]:
test = test.reset_index(drop=False)

In [118]:
col_list = ['us', 'uk', 'jp', 'euro', 'kor', 'ind', 'tw', 'br', 'kor3y', 'kor10y', 'us3y', 'us10y', 'gold'] # list(asset_market.keys())

rename_dict = {
      'kospi':"kor",
      'nasdaq':"us",
      'euro_stoxx':"euro",
      'ftse':"uk",
      'nikkei':"jp",
      'korea_bond_03':"kor3y",
      'korea_bond_10':"kor10y",
      'america_bond_03':"us3y",
      'america_bond_10':"us10y",
      'gold':"gold",
      'brazil':"br",
      'taiwan':"tw",
      'india':"ind"
      }


test.rename(columns=rename_dict, inplace=True)
test = test[col_list]

C:\Users\ceo_pc\AppData\Local\Temp\ipykernel_7456\1635042435.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.rename(columns=rename_dict, inplace=True)


In [125]:
date = '2022-12-29'
wish_date_weight(test,date)

(           date        us       uk        jp     euro     kor           ind  \
 245  2022-12-29  10478.09  7512.72  26093.67  3850.07  2236.4  61133.878906   
 
                tw        br     kor3y   kor10y   us3y  us10y     gold  
 245  14085.019531  110031.0  107370.0  62215.0  81.27  96.18  1814.89  ,
            date        us        uk        jp      euro     kor       ind  \
 245  2022-12-29  0.026705  0.019147  0.066503  0.009812  0.0057  0.155807   
 
            tw        br     kor3y    kor10y      us3y     us10y      gold  
 245  0.035897  0.280428  0.250313  0.145042  0.000189  0.000224  0.004231  )

In [120]:
test = test.reset_index()

In [136]:

pd.options.display.float_format = '{:.5f}'.format
pd.options.display.max_rows = None
np.set_printoptions(precision=6, suppress=True)
# setting the seed allows for reproducible results
np.random.seed(123)

df = pd.read_csv("total_17_22.csv",index_col=0)
test = df[df.index>=('2022')]
lookback_window = 50
lookahead_window = 30
x_test, y_test, portfolio_test_date= make_data(test, lookback_window, lookahead_window)
# portfolio_test_date = x_test['date']
# print(portfolio_test_date)

print(portfolio_test_date)

100%|██████████| 166/166 [00:00<00:00, 8321.83it/s]

['2022-03-22', '2022-03-23', '2022-03-24', '2022-03-25', '2022-03-28', '2022-03-29', '2022-03-30', '2022-03-31', '2022-04-01', '2022-04-04', '2022-04-05', '2022-04-06', '2022-04-07', '2022-04-08', '2022-04-11', '2022-04-12', '2022-04-13', '2022-04-14', '2022-04-15', '2022-04-18', '2022-04-19', '2022-04-20', '2022-04-21', '2022-04-22', '2022-04-25', '2022-04-26', '2022-04-27', '2022-04-28', '2022-04-29', '2022-05-02', '2022-05-03', '2022-05-04', '2022-05-06', '2022-05-09', '2022-05-10', '2022-05-11', '2022-05-12', '2022-05-13', '2022-05-16', '2022-05-17', '2022-05-18', '2022-05-19', '2022-05-20', '2022-05-23', '2022-05-24', '2022-05-25', '2022-05-26', '2022-05-27', '2022-05-30', '2022-05-31', '2022-06-02', '2022-06-03', '2022-06-07', '2022-06-08', '2022-06-09', '2022-06-10', '2022-06-13', '2022-06-14', '2022-06-15', '2022-06-16', '2022-06-17', '2022-06-20', '2022-06-21', '2022-06-22', '2022-06-23', '2022-06-24', '2022-06-27', '2022-06-28', '2022-06-29', '2022-06-30', '2022-07-01', '2022

In [146]:
import pandas as pd
import yfinance as yf

start='2023-01-01'
end='2023-08-31' 
symbol = ['^IXIC','^FTSE','^N225','^STOXX50E','^KS11','^BVSP','^TWII','^BSESN', 'GC=F']
# col = [kospi,nasdaq,euro_stoxx,ftse,nikkei,korea_bond_03,korea_bond_10,america_bond_03,america_bond_10,gold,brazil,taiwan,india]
list1 = yf.download(symbol, start, end)
df = list1['Close']

[*********************100%%**********************]  9 of 9 completed


In [ ]:
  col_list = ['us', 'uk', 'jp', 'euro', 'kor', 'ind', 'tw', 'br', 'kor3y', 'kor10y', 'us3y', 'us10y', 'gold'] # list(asset_market.keys())

  rename_dict = {
      'kospi':"kor",
      'nasdaq':"us",
      'euro_stoxx':"euro",
      'ftse':"uk",
      'nikkei':"jp",
      'korea_bond_03':"kor3y",
      'korea_bond_10':"kor10y",
      'america_bond_03':"us3y",
      'america_bond_10':"us10y",
      'gold':"gold",
      'brazil':"br",
      'taiwan':"tw",
      'india':"ind"
      }